<a id="problema"></a>
# <font color=green>Problem statement</font>

For every aspiring entrepreneur, embarking on the journey of starting a new business presents a myriad of challenges. The most daunting of these challenges is the uncertainty surrounding access to vital information that supports informed decision-making and helps mitigate the risks associated with investing time and money.

In light of this, providing entrepreneurs with tools that furnish organized, accurate, and reliable information would be immensely beneficial. Such tools would offer them a sense of security and heightened confidence in assessing the profitability of their business ideas.

Henceforth, unquestionably, **data science** emerges as the optimal tool for crafting such support tools for entrepreneurs and their nascent businesses.

<a id="preguntas"></a>
# <font color=green>Asking Questions </font>

In accordance with the problem posed above, the following questions arose from both the entrepreneur and the team:

1. What is the best location in Guadalajara, Mexico to open my bicycle business?
2. How many sales will I be able to obtain in the first months of starting my business?
3. What are the products most in demand by potential customers?
4. What prices will be the most competitive for bicycles?
5. How often will a customer want to buy clothing or accessories for their bikes?
6. How many customers will come to my business for a repair or upgrade on their bicycle?

While we cannot guarantee that we will address all the questions raised, we assured the entrepreneur that we will conduct an analysis of the available data. Subsequently, we will evaluate which information can be presented.

# Data Collection
The decision was made to use "Bike Buyers 1000" and "Bike Sales", given that they contain information relevant to the problem being discussed and both datasets complement each other, which were found on kaggle.

"Bike Buyers 1000" 
link: "https://www.kaggle.com/datasets/heeraldedhia/bike-buyers"

"Bike Sales"
link: "https://www.kaggle.com/datasets/liyingiris90/bike-sales"

## Process to obtain the dataset

Since the files contained within the Bike Sales data set are in .xlsx format, the Pandas function used was <font color =red> *pd.read_excel* </font>.

For Bike Buyers, since it is a csv format, our already known function is used  <font color =red> *pd.read_csv*</font>  

In [ ]:
%pip install openpyxl

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from colorama import Fore
from colorama import Style
import matplotlib.pyplot as plt
sns.set()
from scipy.stats import skew, kurtosis
import scipy.optimize as opt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.cluster import KMeans

The respective DataFrames of the data set are saved <font color=blue> "bike sales"</font> and <font color=blue> "bike buyers"

In [ ]:
buyers = pd.read_csv("datasets/bike_buyers.csv")

In [ ]:
bikes = pd.read_excel("datasets/bikes.xlsx")
bikestores = pd.read_excel("datasets/bikeshops.xlsx")
orders = pd.read_excel("datasets/orders.xlsx")

# Dataset Exploration
We proceed to review each DataFrame using <font color=blue> .head() </font> and <font color=blue>.tail()  </font>, as well as <font color=blue> .dtypes, .columns, .shape, .loc[$n:m$] </font> (where $n,m ∈ 𝖭$ and $n<m.$) 

In [ ]:
buyers.dtypes # There are floats that should be integers.
buyers.shape # There are 1000 entries and 13 columns.
buyers.columns # ID, Marital S, Gender, Income, Children, Education, Occupation, Home Owner, Cars, Commute Distance, Region, Age, purchase bike
buyers.head() # NaNs are observed
buyers.tail() # NaNs are observed
c=list(buyers.columns) # The name of the columns is saved in a list, which will be used later to change the names of our dataframe. 
buyers.loc[500:515] # NaNs are observed. 

In [ ]:
bikes.dtypes # The only data that appears to be incorrect is the price, which is of type int. 
bikes.shape # There are a total of 97 rows with 5 columns
bikes.columns # bike.id, model, category1, category2, frame, price. Columns will be renamed.
bikes.head() # NaNs are not observed.
bikes.tail() # NaNs are not observed.
bc=list(bikes.columns)
bikes.loc[46:58] # NaNs are not observed randomly

In [ ]:
bikestores.dtypes # All data has the correct type.
bikestores.shape # There are a total of 30 rows with 6 columns
bikestores.columns # bikeshop.id , bikeshop.name, bikeshop.city, bikeshop.state, latitude , longitude. Columns will be renamed.
bikestores.head() # NaNs are not observed.
bikestores.tail() # NaNs are not observed.
bs=list(bikestores.columns)
bikestores.loc[10:25] # NaNs are not observed randomly. 

In [ ]:
orders.dtypes # It is observed that order.id, order.line, customer.id y product.id They are of type float when they should be of type integer.
orders.shape # There are a total of 15644 rows and 7 columns. 
orders.columns # 'Unnamed: 0', 'order.id', 'order.line', 'order.date', 'customer.id','product.id' y 'quantity'. Columns will be renamed.
oc=list(orders.columns) 
orders.head() # No Nans are observed, on the other hand, it is observed that the column 'Unnamed: 0' is repeated. 
orders.tail() # NaNs are not observed. 
orders.loc[10468:10480] # NaNs are not observed.

Knowing the values of some columns of some dataSets

In [ ]:
orders['product.id'].unique()

In [ ]:
orders['customer.id'].unique()

In [ ]:
bikes['bike.id'].unique()

In [ ]:
bikestores['bikeshop.id'].unique()

# Rename the columns in the DataFrames.

Dictionaries are created to change the name of the columns of our data frames.

In [ ]:
buy_names = {c[0]:'id_buyer',
              c[1]:'civil_status',
              c[2]:'gender',
              c[3]:'salary',
              c[4]:'children',
              c[5]:'education',
              c[6]:"profession",
              c[7]:"own_house",
              c[8]:'cars',
              c[9]:'trip_distance',
              c[10]:'region',
              c[11]:'age',
              c[12]:'purchased_bicycle'}

bike_names = {bc[0]:'id_bicycle',
              bc[1]:'model',
              bc[2]:'category_1',
              bc[3]:'category_2',
              bc[4]:'alloy',
              bc[5]:'price'}

stores_names = {bs[0]:'id_store',
                  bs[1]:'store_name',
                  bs[2]:'city',
                  bs[3]:'state',
                  bs[4]:'latitude',
                  bs[5]:'length'}

orders_names = {oc[1]:'id_order',
                  oc[2]:'order_line',
                  oc[3]:'order_date',
                  oc[4]:'id_store',
                  oc[5]:'id_bicycle',
                  oc[6]:'items_number'}

We notice the fact, that the customer_id column is actually the store_id. Since both match the number of entries and the values associated with each one.

In [ ]:
buyers = buyers.rename(columns=buy_names)
orders = orders.rename(columns=orders_names)
bikes = bikes.rename(columns=bike_names)
stores =  bikestores.rename(columns=stores_names)

In [ ]:
buyers.name = 'buyers'
orders.name = 'orders'
bikes.name = 'bikes'
stores.name = 'stores'

# Removal of NaNs

To have cleaner DataFrames, the NaNs are eliminated

In [ ]:
# The following functions are created to determine NaNs between the different DataFrames.
def nans_numbers(dataframe):
    print(dataframe.isna().sum())

def nans_percentage(dataframe):
    print(dataframe.name)
    print(dataframe.isna().sum()/len(dataframe)*100)
    print("\n")

In [ ]:
nans_percentage(orders)
nans_percentage(stores)
nans_percentage(bikes)
nans_percentage(buyers) # The only DataFrame with NaNs present is the Buyers one.. 
nans_numbers(buyers)

Filling the Nans

In [ ]:
buyers[['children','cars']] = buyers[['children','cars']].fillna(0)
buyers['civil_status'] = buyers['civil_status'].fillna('Single')
buyers['own_house'] = buyers['own_house'].fillna('No')
buyers =buyers.dropna(how='any')
buyers = buyers.reset_index(drop=True) 
nans_numbers(buyers)

# We proceed to change the data type of each DataFrame

For this we will create dictionaries with the new data types and apply the pandas function *.astype()*

In [ ]:
print(orders.name)
print(orders.dtypes) # It has floats that must be integers. 
print("\n")
print(stores.name)
print(stores.dtypes) # Stores has all the correct data types.
print("\n")
print(buyers.name)
print(buyers.dtypes) # It has floats that must be integers. 
print("\n")
print(bikes.name)
print(bikes.dtypes) # It has an int that should be float.  
print("\n")

In [ ]:
c=list(buyers.columns)
oc=list(orders.columns)
new_types_buy = {c[4]:int,
         c[8]:int,
         c[11]:int}

new_types_orders ={oc[1]:int,
                   oc[2]:int,
                   oc[4]:int,
                   oc[5]:int,
                   oc[6]:int}

In [ ]:
orders = orders.astype(new_types_orders)
buyers = buyers.astype(new_types_buy)
bikes['price'] = bikes['price'].astype(float) # This method is used, since you only have to change the data type for one variable.

## We will proceed to eliminate the column 'Unnamed: 0' from the DataFrame orders

In [ ]:
orders.drop(columns='Unnamed: 0',inplace=True)

# We proceed to join the different tables to later make the appropriate aggregations.

### Orders and Bikes.

In [ ]:
or_bikes = pd.merge(orders,bikes,left_on='id_bicycle',right_index=True)
or_bikes.drop(columns=['id_bicycle_x','id_bicycle_y'], inplace=True)
or_bikes

### Stores and Orders

In [ ]:
str_orders = pd.merge(orders,stores,left_on='id_store',right_index=True)
str_orders.drop(columns=['id_store_x','id_store_y'])
str_orders

## Stores, Bikes and Orders

To obtain the true earnings for each bike, a new column called total is created.

In [ ]:
total_sell = or_bikes['items_number']*or_bikes['price']
or_bikes['total'] = total_sell
or_bikes

In [ ]:
str_or_bks = pd.merge(or_bikes,stores,left_on='id_store',right_index=True)
str_or_bks.drop(columns=['id_store_x','id_store_y'],inplace=True)
str_or_bks

# Agregations

## Number of items sold by each store

In [ ]:
str_orders.groupby(['store_name'])['items_number'].sum().sort_values(ascending=False).head(5)

The stores that have sold the most with their respective total are...

>New York Cycles          ~       3471

>Minneapolis Bike Shop      ~     2301

>Las Vegas Cycles            ~    1419

>Columbus Race Equipment      ~   1264

>Albuquerque Cycles            ~  1155

While those with smaller sales are made up of... with a total of...

>Phoenix Bi-peds        ~          246

>Providence Bi-peds      ~       245

>Oklahoma City Race Equipment  ~   234

>Wichita Speed                  ~ 186

>Ann Arbor Speed                ~  128

## Less and Most Sold Bicycles

In [ ]:
or_bikes.groupby('model')['items_number'].sum().sort_values(ascending=False)

The best-selling Bicycles were... with a total of...

>Slice Ultegra        ->              301

>F-Si 3                ->             293

>F-Si Black Inc.         ->           293

>Supersix Evo Ultegra 4    ->         290

>CAAD Disc Ultegra           ->       282


While the least sold were... with a total of...

>Habit 4              ~              143

>Habit Carbon 2        ~             143

>Habit Carbon SE        ~            140

>Synapse Carbon Disc Ultegra D12  ~  139

>Jekyll Carbon 4                ~    132

## Sells obtained by each Bicycle

In [ ]:
or_bikes.groupby('model')['total'].sum().sort_values(ascending=False)

The best and worst sales obtained per bicycle were the following... with a total of...

>Scalpel-Si Black Inc.     ~        3299820.0

>F-Si Black Inc.            ~       3278670.0

>Habit Hi-Mod Black Inc.     ~      2670500.0

>Trigger Carbon 1             ~     2140200.0

>Synapse Hi-Mod Disc Black Inc. ~    2100210.0


While the worst...

>Trail 5    ~                        212715.0

>Catalyst 1  ~                      184710.0

>Catalyst 2   ~                      119925.0

>Catalyst 3    ~                     118080.0

>Catalyst 4     ~                     96695.0

## Sales obtained by each Store

In [ ]:
str_or_bks.groupby('store_name')['total'].sum().sort_values(ascending=False).head(5)

### Let's see that the stores with the highest sales are... with a total of...

>New York Cycles         ~        12476315.0

>Minneapolis Bike Shop    ~        8018850.0

>Columbus Race Equipment   ~       5635205.0

>Las Vegas Cycles           ~      4763175.0

>Albuquerque Cycles          ~     3959000.0

### While those with lower profits...

>Providence Bi-peds                819110.0

>Oklahoma City Race Equipment ~     763390.0

>Phoenix Bi-peds               ~    737905.0

>Wichita Speed                  ~   665510.0

>Ann Arbor Speed                 ~  408680.0

## Sales per order

In [ ]:
or_bikes.groupby('id_order')['total'].sum().sort_values(ascending=False)

## Best cities to sell

In [ ]:
str_or_bks.groupby('city')['total'].sum().sort_values(ascending=False).head(5)

### As we can see below, the best cities are... with a total raised of...

>*New York*   ~      12476315.0

>Minneapolis    ~   8018850.0

>Columbus        ~  5635205.0

>Las Vegas        ~ 4763175.0

>Albuquerque       ~ 3959000.0

### Cities with fewer sales

>Providence   ~      819110.0

>Oklahoma City ~     763390.0

>Phoenix        ~    737905.0

>Wichita         ~   665510.0

>Ann Arbor        ~  408680.0

## First months of sales

### Total Sales per Month

In [ ]:
or_bikes.groupby([pd.Grouper(key='order_date', axis=0, 
                      freq='ME')])['total'].sum().head(5)

In [ ]:
or_bikes.groupby([pd.Grouper(key='order_date', axis=0, freq='ME')])['total'].mean().head(5)

### Total sales for each month of each store

In [ ]:
or_bikes.groupby(['id_store',pd.Grouper(key='order_date', axis=0, 
                      freq='ME')])['total'].agg('sum')

## Alloy and best-selling categories

In [ ]:
or_bikes.groupby(['alloy','category_1','category_2'])['total'].count()

## Obtain Average, Median, Truncated Mean, Standard Deviation, Range and Interquartile, 25th and 75th Percentiles.

In [ ]:
from statistics import mode
from scipy import stats

### Data to be worked on, there are 13 aggregations so far.

In [ ]:
real_buyers = buyers[buyers['purchased_bicycle']=='Yes'].reset_index(drop=True)
real_buyers.drop(columns='purchased_bicycle',inplace=True) # To omit redundancy, the 'purchased_bicycle' column is eliminated
real_buyers

It is decided to obtain estimates of location and variability in the number of items sold by each store, bicycle, profits obtained, and age of bicycle buyers. This is in order to be able to observe whether there are atypical values in our data or not, and which may present a problem when evaluating our regression models.

In [ ]:
items_sold_per_tienda = str_orders.groupby(['store_name'])['items_number'].sum().sort_values(ascending=False)
items_sold_per_bici = or_bikes.groupby('model')['items_number'].sum().sort_values(ascending=False)
gains_per_bici = or_bikes.groupby('model')['total'].sum().sort_values(ascending=False)
gains_per_tienda = str_or_bks.groupby('store_name')['total'].sum().sort_values(ascending=False)
gain_per_order = or_bikes.groupby('id_order')['total'].sum().sort_values(ascending=False)
gain_per_city = str_or_bks.groupby('city')['total'].sum().sort_values(ascending=False)
gain_per_month_from_all = or_bikes.groupby([pd.Grouper(key='order_date', axis=0, freq='ME')])['total'].sum()
gain_per_month_per_tienda = or_bikes.groupby(['id_store',pd.Grouper(key='order_date', axis=0, freq='ME')])['total'].sum();
or_bikes['price']
or_bikes['total'];

## Function for Location and Variability Estimates

In [ ]:
def all_in_one_variability(x):
  print(f'{Fore.CYAN}Mean {Style.RESET_ALL}~ {x.mean()}') 
  print(f'{Fore.CYAN}Median {Style.RESET_ALL}~ {x.median()}') 
  print(f'{Fore.CYAN}Truncated Mean {Style.RESET_ALL}~ {stats.trim_mean(x,.1)}') # Truncated mean of 10% of the data on the sides.
  print(f'{Fore.CYAN}Standard Deviation {Style.RESET_ALL}~ {x.std()}')    
  print(f'{Fore.CYAN}Range {Style.RESET_ALL}~ {x.max() - x.min()}')
  print(f'{Fore.CYAN}Quantil 75 {Style.RESET_ALL}~ {x.quantile(.75)}') 
  print(f'{Fore.CYAN}Quantil 25 {Style.RESET_ALL}~ {x.quantile(.25)}') 
  print(f'{Fore.CYAN}Interquartile Range {Style.RESET_ALL}~ {x.quantile(.75) - x.quantile(.25)}') # Interquartile range

In [ ]:
all_in_one_variability(items_sold_per_tienda)

In [ ]:
all_in_one_variability(items_sold_per_bici)

In [ ]:
all_in_one_variability(gains_per_bici)

In [ ]:
all_in_one_variability(gains_per_tienda)

In [ ]:
all_in_one_variability(gain_per_order)

In [ ]:
all_in_one_variability(gain_per_city)

In [ ]:
all_in_one_variability(gain_per_month_from_all)

In [ ]:
all_in_one_variability(gain_per_month_per_tienda)

In [ ]:
all_in_one_variability(or_bikes['total'])

In [ ]:
all_in_one_variability(real_buyers['age'])

### Customer Salary (Does not imply that they have purchased a bicycle)

In [ ]:
all_in_one_variability(buyers['salary'])

### Bicycle Buyers Salary

In [ ]:
all_in_one_variability(real_buyers['salary'])

## Plots

> Since there is too much data to analyze, a conclusion is left at the end of all our graphs and general findings at the end. However, an analysis is carried out on the data that is considered most relevant for our future models..

> Raw ~ The data that has not undergone changes, such as the Interquartile Range Score (IQR-Score), is used as a reference..

In [ ]:
sns.color_palette("crest", as_cmap=True)

### Function for Density, Histogram and Boxplot Graphs

In [99]:
def graphics_boxplots_density(df,i):
  o=['Items sells per store','Demand for bikes','Profits per bike','Earnings per store','Earnings per order','Earnings per city', 'General sales per month','Sales per month of each store','SubTotal','Customer age','Age of buyers'
  ,'Client Salary','Buyers salary']
  sns.set_theme(style="white")
  fig, axes = plt.subplots(1, 3, figsize=(30, 6), sharex=False, sharey=False)
  sns.boxplot(x=df, ax=axes[0]); # Unfiltered Boxplot
  axes[0].set_title(o[i], fontsize=20)
  axes[0].set_xlabel('',fontsize = 16)
  iqr = df.quantile(0.75) - df.quantile(0.25)
  bottom_filter = df > df.quantile(0.25) - (iqr * 1.5)
  top_filter = df < df.quantile(0.75) + (iqr * 1.5)
  df_filtrado = df[bottom_filter & top_filter]
  c = ' ~ Filtered out ~ IQR'
  b = o[i] + c
  axes[1].set_title(b,fontsize=20)
  axes[1].set_xlabel('' ,fontsize = 16)
  sns.boxplot(df_filtrado,ax=axes[1])
  d = ' ~ Density and Histogram'
  e = o[i] + d
  sns.displot(df_filtrado, bins=16);
  plt.ylabel('Frequency', fontsize=16);
  print(f'{Fore.MAGENTA}Location and Variability Estimates ~ Unfiltered data{Style.RESET_ALL}')
  all_in_one_variability(df)
  print(f'{Fore.GREEN}Kurtosis:{Style.RESET_ALL} {kurtosis(df)}')
  print(f'{Fore.GREEN}Asymmetry: {Style.RESET_ALL}{skew(df)}')
  print(f'{Fore.MAGENTA}Location and Variability Estimates ~ Filtered data ~ Range IQR{Style.RESET_ALL}')
  all_in_one_variability(df_filtrado)#NorbsPR did this.
  print(f'{Fore.GREEN}Kurtosis:{Style.RESET_ALL} {kurtosis(df_filtrado)}')
  print(f'{Fore.GREEN}Asymmetry:{Style.RESET_ALL} {skew(df_filtrado)}')

In [ ]:
graphics_boxplots_density(items_sold_per_tienda,0)

### Demand for bicycles (Number of bicycles sold by model)

> On this occasion it is observed that the values do not change after performing the Interquantile Range Score. Therefore, we will work with the original data.

> We can see in the boxplot that the data accumulates a little towards the right tail. But not significantly.

> After looking at the asymmetry value (0.05) it is suggested that perhaps the distribution is normal, but after seeing the histogram/density graph we realize that our distribution is actually bimodal.

> On the other hand, the kurtosis is -1.31, which indicates that the data is steep, so there is less dispersion of the data and therefore short tails.
> Finally we have a standard deviation of 47.30